In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import joblib
import glob, os


In [12]:
# 데이터 불러오기
column_names = ['timestamp', 'v_raw', 'c_raw', 'voltage', 'current', 'label']
normal_dir = '../realtime/normal/'
arc_dir = '../realtime/arc/'
normal_data = pd.concat([pd.read_csv(f, names=column_names, header=None)
                         for f in glob.glob(os.path.join(normal_dir, '*.csv'))])
arc_data = pd.concat([pd.read_csv(f, names=column_names, header=None)
                      for f in glob.glob(os.path.join(arc_dir, '*.csv'))])
df = pd.concat([normal_data, arc_data]).sample(frac=1).reset_index(drop=True)


In [13]:
# 파생 피처
def add_features(df):
    df['voltage_diff'] = df['voltage'].diff().fillna(0).abs()
    df['current_diff'] = df['current'].diff().fillna(0).abs()
    df['voltage_ma'] = df['voltage'].rolling(5).mean().bfill()
    df['current_ma'] = df['current'].rolling(5).mean().bfill()
    df['power'] = df['voltage'] * df['current']
    df['power_diff'] = df['power'].diff().fillna(0).abs()
    return df

In [14]:
df = add_features(df).dropna()
features = ['voltage', 'current', 'voltage_diff', 'current_diff',
            'voltage_ma', 'current_ma', 'power', 'power_diff']

X_raw = df[features].values
y_raw = df['label'].astype(int).values

In [15]:
# 정규화 및 시퀀스 생성
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)

def create_sequences(data, labels, seq_len=8):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:i+seq_len])
        y.append(labels[i+seq_len - 1])
    return np.array(X), np.array(y)

seq_len = 8
X_seq, y_seq = create_sequences(X_scaled, y_raw, seq_len)
X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq, test_size=0.2, stratify=y_seq, random_state=42)


In [16]:
# GRU 모델 정의
model = Sequential([
    GRU(64, return_sequences=False, input_shape=(seq_len, len(features))),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\NGN Lab\Desktop\kys\논문\전기안전공학\source\arc-detection-project\powerenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
# 학습
model.fit(X_train, y_train, validation_data=(X_val, y_val),
          epochs=50, batch_size=64, verbose=1)

Epoch 1/50


317/317 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8767 - loss: 0.4129 - val_accuracy: 0.8850 - val_loss: 0.3381
Epoch 2/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8863 - loss: 0.3378 - val_accuracy: 0.8856 - val_loss: 0.3292
Epoch 3/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8893 - loss: 0.3230 - val_accuracy: 0.8883 - val_loss: 0.3177
Epoch 4/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8895 - loss: 0.3248 - val_accuracy: 0.8994 - val_loss: 0.3058
Epoch 5/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9022 - loss: 0.3019 - val_accuracy: 0.9024 - val_loss: 0.3029
Epoch 6/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9039 - loss: 0.2974 - val_accuracy: 0.9045 - val_loss: 0.2995
Epoch 7/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9037 - loss: 0.3028 - val_accuracy: 0.9049 - val_loss: 0.2991
Epoch 8/50
317/317 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9028 - loss: 0.3037 - val_accuracy: 0.9039 - val_

In [18]:
# 저장
model.save('./model/gru_model.h5')
joblib.dump(scaler, './model/scaler_gru.joblib')

['./model/scaler_gru.joblib']

In [19]:
import tensorflow as tf
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

model = tf.keras.models.load_model('./model/gru_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 핵심 설정 🔽
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,        # 기본 연산
    tf.lite.OpsSet.SELECT_TF_OPS           # TensorFlow 연산 허용
]
converter._experimental_lower_tensor_list_ops = False  # TensorList 연산 내림 방지

# 최적화 옵션 (선택)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 변환
tflite_model = converter.convert()

# 저장
with open('./model/gru_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ GRU 모델 TFLite 변환 성공 (SELECT_TF_OPS 포함)")


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmp2dsebqvx\assets


INFO:tensorflow:Assets written to: C:\Users\NGNLAB~1\AppData\Local\Temp\tmp2dsebqvx\assets


Saved artifact at 'C:\Users\NGNLAB~1\AppData\Local\Temp\tmp2dsebqvx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 8, 8), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1987957287312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957287504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957281936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957284624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957286736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957286928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1987957286352: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ GRU 모델 TFLite 변환 성공 (SELECT_TF_OPS 포함)
